# RANK 1 CONSTRAIN SYSTEMS

In [1]:
import numpy as np

In [2]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.5 MB/s eta 0:00:00


## TRANSFORMING EQUATIONS INTO R1CS

In [3]:
# r = y*z
# [1,r,y,z]

O1 = np.matrix([[0,1,0,0]])
L1 = np.matrix([[0,0,1,0]])
R1 = np.matrix([[0,0,0,1]])

a1 = np.array([1, 24, 3, 8])
print ( np.matmul(O1, a1) == np.matmul(L1, a1) * np.matmul(R1, a1) )

[[ True]]


In [4]:
# r = x*y*z
# v = x*y , r = v*z
# [1,r,x,y,z,v]

O2 = np.matrix ( [[0,0,0,0,0,1],
              [0,1,0,0,0,0]] )
L2 = np.matrix ( [[0,0,1,0,0,0],
              [0,0,0,0,0,1]] )
R2 = np.matrix ( [[0,0,0,1,0,0],
              [0,0,0,0,1,0]] )

a2 = np.array( [1, 24, 2, 3, 4, 6] )
print ( np.matmul(O2, a2) == np.multiply ( np.matmul(L2, a2) , np.matmul(R2, a2) ) )

[[ True  True]]


In [5]:
# r = w*x*y*z
# u = w*x , v = y*z , r = u*v
# [1,r,w,x,y,z,u,v]

O3 = np.matrix ( [[0,0,0,0,0,0,1,0],
               [0,0,0,0,0,0,0,1],
               [0,1,0,0,0,0,0,0]] )
L3 = np.matrix ( [[0,0,1,0,0,0,0,0],
               [0,0,0,0,1,0,0,0],
               [0,0,0,0,0,0,1,0]] )
R3 = np.matrix ( [[0,0,0,1,0,0,0,0],
               [0,0,0,0,0,1,0,0],
               [0,0,0,0,0,0,0,1]] )

a3 = np.array( [1, 120, 2, 3, 4, 5, 6, 20] )
print ( np.matmul(O3, a3) == np.multiply ( np.matmul(L3, a3) , np.matmul(R3, a3) ) )

[[ True  True  True]]


In [6]:
# r = y*z + 1

O4 = np.matrix([[-1,1,0,0]])
L4 = np.matrix([[0,0,1,0]])
R4 = np.matrix([[0,0,0,1]])

a4 = np.array([1, 25, 3, 8])
print ( np.matmul(O4, a4) == np.matmul(L4, a4) * np.matmul(R4, a4) )

[[ True]]


In [7]:
# r = 2*( y**2 ) + z + 2
# r-z-2 = (2*y) * (y)
# # [1,r,y,z]

O5 = np.matrix([[-2,1,0,-1]])
L5 = np.matrix([[0,0,2,0]])
R5 = np.matrix([[0,0,1,0]])

a5 = np.array([1, 28, 3, 8])
print ( np.matmul(O5, a5) == np.matmul(L5, a5) * np.matmul(R5, a5) )

[[ True]]


## LAGRANGE INTERPOLATION

In [8]:
from scipy.interpolate import lagrange
x_values = [1, 2, 3, 4]
y_values = [4, 8, 2, 1]

print(lagrange(x_values, y_values))

     3      2
2.5 x - 20 x + 46.5 x - 25


In [9]:
import galois
import numpy as np

GF17 = galois.GF(17)

x = GF17(np.array([1,2,3,4]))
y = GF17(np.array([4,8,2,1]))

p = galois.lagrange_poly(x, y)

print( p(1) == GF17(4) )
print( p(2) == GF17(8) )
print( p(3) == GF17(2) )
print( p(4) == GF17(1) )

True
True
True
True


## SCHWARTZ - ZIPPEL  LEMMA

In [10]:
# Using the Schwartz-Zippel Lemma to test if two vectors are equal
# In a finite field F, we pick a random value u from [0,p). Then we evaluate yf = f(u) and yg = g(u)
# If yg = yf, we assume that both f and g are equal.

p = 103
GF = galois.GF(p)

x = GF(np.array([1,2,3]))
v1 =  GF(np.array([4,8,19]))
v2 =  GF(np.array([4,8,19]))

p1 = galois.lagrange_poly(x, v1)
p2 = galois.lagrange_poly(x, v2)

import random
u = random.randint(0, p)
lhs = p1(u)
rhs = p2(u)

print( lhs == rhs )

True


## QUADRATIC ARITHMETIC PROGRAMS

In [11]:
# Vector addition is homomorphic to polynomial addition
# Scalar multiplication is repeated addition only

p = 17
GF = galois.GF(p)
c = GF(10)

xs = GF(np.array([1,2,3]))
v1 =  GF(np.array([4,8,2]))
v2 =  GF(np.array([1,6,12]))

def L(v):
    return galois.lagrange_poly(xs, v)

print( L(v1 + v2) == L(v1) + L(v2) )
print( L( c * v1 ) == c * L(v1) )

True
True


In [12]:
# Succintly testing that A v1 = B v2
# A = [[6,3],[4,7]] , B = [[3,9],[12,6]]
# v1 = [2,4] , v2 = [2,2]


import random

p = 17
GFp = galois.GF(p)

x = GF(np.array([1, 2]))
def L(v):
    return galois.lagrange_poly(x, v)

p1 = L(GF(np.array([6, 4])))
p2 = L(GF(np.array([3, 7])))
q1 = L(GF(np.array([3, 12])))
q2 = L(GF(np.array([9, 6])))

# Transformation of matrices A and B into column vectors.

print(p1)
# 15x + 8 (mod 17)
print(p2)
# 4x + 16 (mod 17)
print(q1)
# 9x + 11 (mod 17)
print(q2)
# 14x + 12 (mod 17)

# Verification using Schwartz-Zippel Lemma
u = random.randint(0, p)
t = GF(u)

left_hand_side = p1(t) * GF(2) + p2(t) * GF(4)
right_hand_side = q1(t) * GF(2) + q2(t) * GF(2)

print( left_hand_side == right_hand_side )

15x + 8
4x + 16
9x + 11
14x + 12
True


## R1CS  TO  QAP

In [13]:
gf23 = galois.GF(23)

# r = w*x*y*z
# u = w*x , v = y*z , r = u*v
# [1,r,w,x,y,z,u,v]

O6 = np.matrix ( [[0,0,0,0,0,0,1,0],
               [0,0,0,0,0,0,0,1],
               [0,1,0,0,0,0,0,0]] )
L6 = np.matrix ( [[0,0,1,0,0,0,0,0],
               [0,0,0,0,1,0,0,0],
               [0,0,0,0,0,0,1,0]] )
R6 = np.matrix ( [[0,0,0,1,0,0,0,0],
               [0,0,0,0,0,1,0,0],
               [0,0,0,0,0,0,0,1]] )

# Converting them to field arrays simply by wrapping them with GF
O6 = gf23( (O6 + 23) % 23 )
L6 = gf23( (L6 + 23) % 23 )
R6 = gf23( (R6 + 23) % 23 )

w = gf23(2); x = gf23(3); y = gf23(4); z = gf23(5)
u = w*x; v = y*z
r = u*v

# Creating a valid witness
witness = gf23(np.array( [gf23(1),r,w,x,y,z,u,v] ) )
print ( np.equal( np.matmul(L6, witness) * np.matmul(R6, witness), np.matmul(O6, witness) ) )

[ True  True  True]


In [14]:
# Turning each of the columns of the matrices into a list of galois polynomials that interpolate the columns.

def interpolate_column(col):
    xs = gf23(np.array([1,2,3]))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns.
# apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L6)
V_polys = np.apply_along_axis(interpolate_column, 0, R6)
W_polys = np.apply_along_axis(interpolate_column, 0, O6)

print(U_polys)
print(V_polys)
print(W_polys)

[Poly(0, GF(23)) Poly(0, GF(23)) Poly(12x^2 + 9x + 3, GF(23))
 Poly(0, GF(23)) Poly(22x^2 + 4x + 20, GF(23)) Poly(0, GF(23))
 Poly(12x^2 + 10x + 1, GF(23)) Poly(0, GF(23))]
[Poly(0, GF(23)) Poly(0, GF(23)) Poly(0, GF(23))
 Poly(12x^2 + 9x + 3, GF(23)) Poly(0, GF(23))
 Poly(22x^2 + 4x + 20, GF(23)) Poly(0, GF(23))
 Poly(12x^2 + 10x + 1, GF(23))]
[Poly(0, GF(23)) Poly(12x^2 + 10x + 1, GF(23)) Poly(0, GF(23))
 Poly(0, GF(23)) Poly(0, GF(23)) Poly(0, GF(23))
 Poly(12x^2 + 9x + 3, GF(23)) Poly(22x^2 + 4x + 20, GF(23))]


In [16]:
# Computing h(x)

from functools import reduce

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

u = inner_product_polynomials_with_witness(U_polys, witness)

v = inner_product_polynomials_with_witness(V_polys, witness)

w = inner_product_polynomials_with_witness(W_polys, witness)

# t = (x - 1)(x - 2)(x - 3)(x - 4) since there are four rows.
t = galois.Poly([1, 22], field = gf23) * galois.Poly([1, 21], field =gf23 ) * galois.Poly([1, 20], field = gf23) * galois.Poly([1, 19], field = gf23)

h = (u * v - w) // t

print ( u * v == w + h * t )


False
